In [1]:
from bs4 import BeautifulSoup
import json
from pymongo import MongoClient
import re
import requests
from time import sleep

In [2]:
err_log = open('log.err', 'a')
f = open('../data/beer_pages.json')
j = json.loads(f.read())
beer_pages = j

In [3]:
client = MongoClient()
db = client.ratebeer_database

beer_col = db.unique_beers # though some beers seem to repeat as different editions or as draft version, etc
rev_col = db.reviews

In [ ]:
urls = []
for b in beer_col.find():
    urls.append(b['urlid'])

len(urls)

len(beer_pages)

new_urls = list(set(beer_pages) - set(urls))

In [15]:

def getName(s):
    return s.find('div', {'class': 'user-header'}).text
def brewery(s):
    return s.find('a', {'id':'_brand4'}).find('span', {'itemprop': 'name'}).text
def overall_rating(s):
    return s.find('div', {'class': 'big-circle'}).find('div', {'class': 'ratingValue'}).text
def style(s):
    return s.find(text=re.compile('Style')).findNextSibling().text
def avg_score(s):
    return s.find('a', {'name':'real average'}).find('span', {'itemprop': 'ratingValue'}).text
def cals(s):
    return s.find('abbr', {'title':'Estimated calories for a 12 fluid ounce serving'}).findNext('big').text
def abv(s):
    return s.find('abbr', {'title':'Alcohol By Volume'}).findNext('big').text
def description(s):
    return s.find('span', {'id':'_description3'}).text
def usrid(u):
    return u.text.split('(')[-1].strip(')')
def rating(u):
    return u.findPrevious('div').text
def cat_ratings(u):
    return u.findPrevious('strong').text
def review(u):
    return u.findNext('div').text
def get_date(u):
    return u.parent.text.split('-')[-1]

search_dict = {
    'name' : getName,
    'brewery' : brewery,
    'overall_rating' : overall_rating,
    'style' : style,
    'avg_score' : avg_score,
    'calories_12_oz' : cals,
    'abv' : abv,
    'description' : description
}

usr_search_dict = {
    'usrid' : usrid,
    'rating' : rating,
    'category_ratings' : cat_ratings,
    'review' : review,
    'date' : get_date   
}

In [ ]:
for page in beer_pages:         
    b_url = 'https://www.ratebeer.com' + page
    resp = requests.get(b_url)
    soup = BeautifulSoup(resp.text,'lxml')
    beer_dict={}
    try:
        last_page = soup.findAll('a', {'class':'ballno'})[-1].text
    except:
        last_page = 1
    soup = soup.find('div',{'id':'container'})

    for k in search_dict:
        try:
            beer_dict[k] = search_dict[k](soup)
        except:
            beer_dict[k] = 'none'
            log_err(b_url,k)
    
    beer_col.insert({'urlid' : page, 'content': beer_dict})

    for i in range(1,int(last_page)+1):
        sleep(.1)
        url = b_url + '1/{}/'.format(i)

        resp = requests.get(url)
        soup = BeautifulSoup(resp.text,'lxml')

        soup = soup.find('div',{'id':'container'})
       
        try:
            table = soup.find('div', {'class': 'reviews-container'})
        except:
            log_err(url,'no table')
            continue
            
        try: 
            usrs = table.findAll('a', {'href': re.compile('user')})
        except:
            continue
            
        for usr in usrs:   
            review_dict={}
            review_dict['beer_url'] = b_url
            for k in usr_search_dict:
                try:
                    review_dict[k] = usr_search_dict[k](usr)
                except:
                    review_dict[k] = 'none'
                    log_err(url,k)
            rev_col.insert(review_dict)


/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:50: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [16]:
def log_err(url,k):
    err_log.write(url + ',' +  k + '\n')
    

In [19]:
err_log.close()
